
#### Chatbot with Retrieval-Augmented Generation (RAG)
#### This notebook demonstrates how to set up and use the chatbot package, integrating the `microsoft/Phi-3-mini-4k-instruct` model with LangChain and Retrieval-Augmented Generation (RAG).

#### Requirements
#### To start, you'll need to install the required libraries:

In [ ]:

!pip install torch transformers langchain sentence-transformers faiss-cpu datasets

In [ ]:
class CustomRetriever:
    def __init__(self, model_name="all-MiniLM-L6-v2"):
        self.embedder = SentenceTransformer(model_name)
        self.index = faiss.IndexFlatL2(self.embedder.get_sentence_embedding_dimension())
        self.documents = []

    def add_documents(self, docs):
        embeddings = self.embedder.encode(docs)
        self.index.add(embeddings)
        self.documents.extend(docs)

    def retrieve(self, query, top_k=5):
        query_embedding = self.embedder.encode([query])
        distances, indices = self.index.search(query_embedding, top_k)
        return [self.documents[i] for i in indices[0]]


In [ ]:
class RAGPipeline:
    def __init__(self, model, tokenizer, retriever):
        self.model = model
        self.tokenizer = tokenizer
        self.retriever = retriever

    def generate_response(self, query):
        retrieved_docs = self.retriever.retrieve(query)
        context = " ".join(retrieved_docs)
        inputs = self.tokenizer.encode(context + query, return_tensors="pt")
        outputs = self.model.generate(inputs, max_length=200)
        response = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return response


In [ ]:
# Initialize the retriever and add documents
retriever = CustomRetriever()
retriever.add_documents(["Document 1: AI advancements in healthcare", 
                         "Document 2: Generative models in NLP", 
                         "Document 3: The future of machine learning"])

# Load model and tokenizer
model_name = "microsoft/Phi-3-mini-4k-instruct"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

# Create the RAG pipeline
rag_pipeline = RAGPipeline(model, tokenizer, retriever)


#### Fine-tuning

In [ ]:
from transformers import Trainer, TrainingArguments
from datasets import load_dataset

def fine_tune_model(model, dataset_path, tokenizer):
    # Load dataset
    dataset = load_dataset("json", data_files=dataset_path)["train"]

    def preprocess_function(examples):
        inputs = tokenizer(examples["input"], truncation=True, padding="max_length", max_length=256)
        labels = tokenizer(examples["output"], truncation=True, padding="max_length", max_length=256).input_ids
        inputs["labels"] = labels
        return inputs

    tokenized_dataset = dataset.map(preprocess_function, batched=True)
    training_args = TrainingArguments(output_dir="./results", per_device_train_batch_size=4, num_train_epochs=3)
    trainer = Trainer(model=model, args=training_args, train_dataset=tokenized_dataset)
    trainer.train()
    return model
